In [89]:
import numpy as np
from typing import Type, Tuple

In [90]:
np.random.seed(0)

In [123]:
class ActivationFunction:
    def __init__(self, type: str) -> None:
        self.type = type
        
    def forward(z: np.ndarray) -> np.ndarray:
        pass

class Layer:
    def __init__(self, type: str, size: Tuple, activation: Type[ActivationFunction] | None) -> None:
        self.type: str = type
        self.size: Tuple = size
        self.activation = activation
        
    def __str__(self) -> str:
        return f"Base Layer class"

    def forward(self):
        pass

class Dense(Layer):
     
    def __init__(self, size: Tuple, activation: Type[ActivationFunction]) -> None:
        super().__init__("Dense", size, activation)
        self.weights = np.random.rand(*size)
        self.bias = np.random.randint(-5, 5)
        
    def __str__(self) -> str:
        return f"- {self.type} -\nWeights:\n{self.weights}\n{'-'*15}\nBias: {self.bias}"


    def forward(self, x: np.ndarray) -> np.ndarray:
        return np.dot(x, self.weights) + self.bias


class Input(Layer):
    def __init__(self, inputs: np.ndarray) -> None:
        super().__init__("Input", inputs.size, None)
        self.x: np.ndarray = inputs
        
    def forward(self) -> np.ndarray:
        return self.x
        
  
class ReLu(ActivationFunction):
    def __init__(self, type: str) -> None:
        super().__init__("Relu")
        
    def forward(z: np.ndarray) -> np.ndarray:
        return np.maximum(0, z)
    
    
class Sigmoid(ActivationFunction):
    def __init__(self, type: str) -> None:
        super().__init__("Sigmoid")
        
    def forward(z: np.ndarray):
        return 1 / (1 + np.exp(-z))

    
class NeuralNetwork:
    def __init__(self) -> None:
        self.layers: np.ndarray[Type[Layer]] = np.array([])
    
    def __str__(self) -> str:
        return ",\n".join([f"Layer {layer.type} (size={layer.size})" for layer in self.layers])
    
    def add(self, layer: Type[Layer]) -> None:
        self.layers = np.append(self.layers, layer)
        
    def forward(self) -> np.ndarray:
        z = self.layers[0].forward()
        for layer in self.layers[1:]:
            z = layer.forward(z)
            
            if layer.activation:
                z = layer.activation.forward(z)
                            
        return z

In [134]:
nn = NeuralNetwork()
nn.add(Input(inputs=np.random.rand(2)))
nn.add(Dense(size=(2, 3), activation=ReLu))
nn.add(Dense(size=(3, 3), activation=Sigmoid))


nn.forward()

array([0.92697234, 0.81727546, 0.88744932])